# 隐马尔可夫模型

隐马尔可夫模型(HiddenMarkovModel, HMM)是一种带有隐含状态的基于统计方法的马尔可夫模型
，在有状态的智能系统中应用较多。它以贝叶斯网络作为数据框架，用Baum-Welch算法进行无监
督训练，通过Viterbi算法计算隐含状态。

实现请参考hmmLearn
https://hmmlearn.readthedocs.io/en/latest/index.html

从模型角度看，隐马尔可夫模型是一个由隐藏层和可见层组成的两层模型，底层隐藏层由马尔可夫
状态链组成，可见层由普通状态结点组成。隐藏层状态需要用可见层状态去推测。

解决的问题分三类：
1. 状态问题。已知模型所有参数（隐藏层状态类型，可见层状态类型，输出概率表，转换概率表）
，求任意一个可见层状态链产生的几率。
2. 解码问题，即猜测隐藏层状态。已知模型所有参数，并知道某一组可见层的状态链，求这些
可见状态下对应的隐藏状态是什么。
3. 训练模型参数。已知隐藏层状态数量、可见层状态数量、足够长的可见状态链，求模型的
输出概率表和转换概率表。

In [7]:
# 先解决第二类问题

import numpy as np
import hmmlearn.hmm as hmm  

emission_probability = np.array([[0.4,0.3,0.3],[0.2,0.3,0.5],[0.1,0.8,0.1]])
transition_probability = np.array([[0.7,0.2,0.1],[0.3,0.5,0.2],[0.3,0.4,0.3]])
start_probability = np.array([0.5,0.3,0.2]) #初始概率，即隐藏层状态链第一个结点的状态概率分布

model = hmm.MultinomialHMM(n_components=3) # 三种可见状态类型
model.startprob_ = start_probability
model.transmat_ = transition_probability
model.emissionprob_ = emission_probability

observe_chain = np.array([0,2,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,1,1,0]).reshape(-1,1)
print(model.predict(observe_chain))
print(model.score(observe_chain))

[0 0 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 0 0 0]
-20.81800548939279


## 针对问题一——可见层状态链可能性评估

算法可以是暴力求解、前向算法或后向算法，后两者合称forward-backward algorithm

### 1. 暴力求解
- 计算每一种隐藏状态链的出现概率
- 针对每一种隐藏状态链，计算在其出现的情况下给定可见状态链的出现概率
- 所有隐藏状态链下计算的给定可见状态链概率求和即得到解

时间复杂度是$O(T·N^T)$，隐藏层状态数是N，可见层结点数量是T

### 2. 前向算法

动态规划法（空间换时间）改造前面的暴力求解过程，算法时间复杂度降低到$O(T·N^2)$。

从前往后，逐层计算状态概率值，不断向下一个状态转移，直至最后一个隐藏状态，然后叠加所有相同路径，计算总和。(充分利用前置相同的马尔科夫链转移结果，减少计算次数)

### 3. 后向算法

和前向算法区别是，从后往前计算，不断递归，也是使用动态规划法。时间复杂度也是$O(T·N^2)$

## 针对问题二

### Viterbi算法

给定HMM模型参数和一个可见状态链，找出可能的隐藏状态链。
两个步骤：
1. 前向推算，用动态规划法从第一个隐藏状态逐步计算到最后一个状态，并记录在每个状态出现的概率和概率最大的前一个结点状态
2. 反向推理，根据最后一个最大概率的结点状态，不断根据1中记录的概率最大的前一个结点状态，递归获得最可能的隐藏序列

In [10]:
X = np.array([0,2,1,1,1,1,1,2,2,2,0]).reshape(-1,1)
print(model.predict(X,[4,7]))
print(model.score(observe_chain))

[0 0 0 0 0 0 0 0 0 0 0]
-20.81800548939279


## 针对问题三

### EM算法
- 定义系统评价方法或目标函数
- 随机初始化模型参数$\theta$
- E(Expectation)步：用当前参数$\theta$评估所有输入数据
- M(Maximization)步：用评估结果重新计算$\theta$参数，使得新$\theta$对输入数据获得更好的评价

算法局限性：不能找到全局最优参数，容易陷入局部最优

但是EM仍然被广泛使用，原因是：
1. 这是一种全局最优和算法复杂度的权衡，如果真的要找全局最优往往时间复杂度超过多项式级
2. 在高维环境中，算法陷入局部最小值的概率较小

### Baum-Welch算法
HMM前两类问题都假设模型参数已给定，但是参数本身需要求解。一般有两个思路：
1. 由足够多的隐藏层状态链表生成转换概率矩阵，有足够多输出数据生成输出概率矩阵
2. 给出足够多的可见状态链表，根据其猜测输出概率和转移概率矩阵

第一种需要人工整理和标注，第二种就是Baum-Welch算法（其实就是EM算法）。

具体公式推导参考https://www.cnblogs.com/crackpotisback/p/8670054.html

## 连续型概率分布

前面说的都是离散型概率分布，所有常用多项式分布，即hmmLearn中的MultinomialNB。
解决连续型概率分布常用多元高斯分布或高斯混合分布，即hmmLearn中的GaussianHMM和GMMHMM。